In [218]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import random as rand
import queue
import csv
from collections import OrderedDict
from IPython.display import clear_output
import csv
from heapq import merge
from sklearn import preprocessing
from operator import itemgetter
import bisect

from sklearn.datasets import make_circles, make_moons
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from IPython.core.debugger import set_trace
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

In [242]:
rand.seed(37)
AAMass = {'A':71.037114, 'C':103.009185, 'D':115.026943, 'E':129.042593, 'F':147.068414, 'G':57.021464, 'H':137.058912,
          'I':113.084064, 'K':128.094963, 'L':113.084064, 'M':131.040485, 'N':114.042927, 'P':97.052764, 'Q':128.058578,
          'R':156.101111, 'S':87.032028, 'T':101.047679, 'V':99.068414, 'W':186.079313, 'Y':163.0633}

H2O = 18.01528
PROTON = 1.00727647
specsize = 8000
precursor_tolerance = 200.0
num_psm = 1
charge = 2
use_mods = True

In [243]:
def GetAAMass(AA):
    return AAMass[AA] + 57.021464 if AA == 'C' else AAMass[AA]

def GetPepMass(pep):
    return sum(AAMass[aa] for aa in pep) + H2O

def GetSpectrum(Seq):
    size = len(Seq)
    outsize = 2*size
    bspectrum = []
    yspectrum = []
    bspectrum.append(GetAAMass(Seq[0]) + PROTON)
    yspectrum.append(GetAAMass(Seq[-1]) + H2O + PROTON)
    
    for (i, fAA), bAA in zip(enumerate(Seq[1:]), Seq[-2::-1]):
        bspectrum.append(bspectrum[i] + GetAAMass(fAA))
        yspectrum.append(yspectrum[i] + GetAAMass(bAA))
    
    mergedout = list(merge(bspectrum, yspectrum))
    if mergedout[-1] > specsize:
        print(mergedout[-1])
        print(Seq)
    tspec = np.zeros(specsize)
    tspec[np.rint(mergedout).astype(int)] = 1
    return tspec

by = GetSpectrum('ACDEFG')
print('printing by:\n' + str(by))
print(by.shape)
print(AAMass['L'])

printing by:
[0. 0. 0. ... 0. 0. 0.]
(8000,)
113.084064


In [270]:
pepidmass = {}
mods = {}
spectra = []
sortedpeps = []
specmasses = []

f=open("human_consensus_final_true_lib_id.msp", "r")
#f=open("test_id.msp", "r")
#fo = open('output.csv', 'w')
#fo.write('Q,P,N\n')
fixedlen = 300
dataset = []
label = []
lines = f.readlines()
count = 0
limit = 100000
pep = mass = numPeaks = 0
#spec = []
isName = isMW = isNumPeaks = isID = False
new = prev = 0
maxpeaks = maxmoz = 0
ids = []
i = 0
mod_count = 0
multimods = 0
while i < len(lines) and limit > 0:
    line = lines[i]
    i += 1
    
    splits = line.split(':')
    if splits[0] == 'ID':
        pepid = int(splits[1])
        isID = True
    
    if isID and splits[0] == 'Name' and '_' in splits[1]:
        split1 = splits[1]
        l_charge = int(split1[split1.find('_') - 1])
        if int(split1[split1.find('_') + 1]) > 7:
            multimods += 1
        if l_charge != charge:
            isID = isName = isMW = isNumPeaks = False
            continue
        if use_mods:
            if '(' in split1 and ')' in split1:
                mod_count += 1
                mod = split1.split('_')[1].strip()[:-1]
                if mod not in mods:
                    mods[mod] = 1
                else:
                    mods[mod] += 1
            pep = split1.split('/')[0].lstrip(' ')
            isName = True
        elif '(' not in split1 and ')' not in split1:
            pep = split1.split('/')[0].lstrip(' ')
            isName = True
        
    if isID and isName and splits[0] == 'MW':
        mass = float(splits[1])
        if round(mass) < specsize:
            isMW = True
            limit = limit - 1
        else:
            isID = isName = isMW = isNumPeaks = False
            continue
        
    if isID and isName and isMW and splits[0] == 'Num peaks':
        numPeaks = int(splits[1])
        if numPeaks > maxpeaks:
            maxpeaks = numPeaks
            
        spec = np.zeros(specsize)
        while (lines[i] != '\n'):
            mzline = lines[i]
            i += 1
            mzsplits = mzline.split('\t')
            moz, intensity = float(mzsplits[0]), float(mzsplits[1])
            if moz > maxmoz:
                maxmoz = moz
            spec[round(moz)] += round(intensity)
        
        #padlen = fixedlen - len(spec)
        #leftpad = rand.randint(0, padlen)
        #rightpad = padlen - leftpad
        #npspec = np.reshape(spec, (len(spec), 2))
        #npspec = np.pad(npspec, ((leftpad, rightpad), (0, 0)), 'constant', constant_values=(0, 0))
        spec = np.clip(spec, None, 1000.0)
        spec = preprocessing.scale(spec)
        #min_max_scaler = sklearn.preprocessing.MinMaxScaler()
        #npspec[:,1] = sklearn.preprocessing.minmax_scale(npspec[:,1])
        
        isNumPeaks = True
    if isID and isName and isMW and isNumPeaks:
        isID = isName = isMW = isNumPeaks = False
        spectra.append([spec, mass])
        ids.append(pepid)
        if pep not in pepidmass:
            pepidmass[pep] = [[pepid], GetPepMass(pep)] #add calculated mass
        else:
            pepidmass[pep][0].append(pepid)
            
        #add calculated mass
        #revPep = pep[0] + pep[1:-1][::-1] + pep[-1]
        #if pep == revPep:
        #    print('decoy is the same. shuffling')
        #    revPep = ''.join(rand.sample(revPep,len(revPep)))
        
        #fo.write('"' + str(list(spec)).strip('[]') + '",' + str(pep) + ',' + str(revPep) + '\n')
        #tspec = GetSpectrum(pep)
        #rtspec = GetSpectrum(revPep)
        #padlen = fixedlen - len(tspec)
        #leftpad = rand.randint(0, padlen)
        #rightpad = padlen - leftpad
        #tspec = np.pad(tspec, ((leftpad, rightpad), (0, 0)), 'constant', constant_values=(0, 0))
        #padlen = fixedlen - len(rtspec)
        #leftpad = rand.randint(0, padlen)
        #rightpad = padlen - leftpad
        #rtspec = np.pad(rtspec, ((leftpad, rightpad), (0, 0)), 'constant', constant_values=(0, 0))
        
        #spectensor = transforms.ToTensor()(spec).view(specsize,-1).float()
        #tspectensor = transforms.ToTensor()(tspec).view(specsize,-1).float()
        #rtspectensor = transforms.ToTensor()(rtspec).view(specsize,-1).float()
        
        #dataset.append([spec, tspec, rtspec])
        #label.append([1, -1])
        #dataset.append([np.reshape(spec, (len(spec), 2)), tspec, rtspec])
        count = count + 1
        pepid = pep = mass = numPeaks = 0
        spec = []
        new = int((i/len(lines)) * 100)
        if new > prev:
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new
print('num multimods: ' + str(multimods))

54%
num multimods: 1


In [271]:
sortedpeps = [[pep, float(mass[1])] for [pep, mass] in pepidmass.items()]
sortedpeps, sortedpepmasses = list(zip(*sorted(sortedpeps, key=lambda pair: pair[1])))
#sortedspectra, sortedspecmasses = list(zip(*sorted(spectra, key=lambda pair:pair[1])))
sortedspectra, sortedspecmasses = list(zip(*spectra))
#sortedpeps.sort(key=itemgetter(1))
print('num modified specs: ' + str(sum(mods.values())))
print('num unique mods: ' + str(len(mods)))

print('max peaks: ' + str(maxpeaks))
print('count: ' + str(count))
print('max moz: ' + str(maxmoz))
f.close()
#for it in mods.items():
#    print(it)

#fo.close()

num modified specs: 28747
num unique mods: 1629
max peaks: 251
count: 99999
max moz: 2370.2


In [272]:
print(len(ids))
print(len(spectra))
print(len(pepidmass))
print(len(sortedpeps))
print(sortedpeps[0:10])
print(sortedpepmasses[0:10])
print(sortedspectra[0:10])
print(sortedspecmasses[0:10])

99999
99999
88212
88212
('CLLPK', 'CSVIR', 'CMCIK', 'LILLK', 'LAATPK', 'LAQLR', 'INLLK', 'GLVIAK', 'IEVLK', 'LDILK')
(572.34032, 576.310082, 596.253162, 598.446499, 599.368978, 599.380211, 599.405362, 599.405363, 600.389378, 600.389378)
(array([-0.11289521, -0.11289521, -0.11289521, ..., -0.11289521,
       -0.11289521, -0.11289521]), array([-0.13213208, -0.13213208, -0.13213208, ..., -0.13213208,
       -0.13213208, -0.13213208]), array([-0.07854022, -0.07854022, -0.07854022, ..., -0.07854022,
       -0.07854022, -0.07854022]), array([-0.13432618, -0.13432618, -0.13432618, ..., -0.13432618,
       -0.13432618, -0.13432618]), array([-0.12407095, -0.12407095, -0.12407095, ..., -0.12407095,
       -0.12407095, -0.12407095]), array([-0.09115811, -0.09115811, -0.09115811, ..., -0.09115811,
       -0.09115811, -0.09115811]), array([-0.10909656, -0.10909656, -0.10909656, ..., -0.10909656,
       -0.10909656, -0.10909656]), array([-0.09858578, -0.09858578, -0.09858578, ..., -0.09858578,
     

In [273]:
#spectra = np.reshape(spectra, (len(spectra), 2))
#sortedpeps = np.reshape(sortedpeps, (len(sortedpeps), 2))
#print(spectra.shape)
#print(sortedpeps.shape)
sortedtspecs = [preprocessing.scale(GetSpectrum(pep)) for pep in sortedpeps]
#print(sortedtspecs.shape)

In [274]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        #self.conv1 = nn.Conv2d(1, 64, 7)
        self.linear1_1 = nn.Linear(specsize, 1024)
        self.linear1_2 = nn.Linear(1024, 512)
        self.linear1_3 = nn.Linear(1024, 512)
        #self.linear1_3 = nn.Linear(256, 128)
        self.linear1_4 = nn.Linear(512, 256)
        
        self.lstm2_1 = nn.Linear(specsize, 1024)
        self.linear2_1 = nn.Linear(1024, 512)
        self.linear2_2 = nn.Linear(512, 256)
        #self.pool1 = nn.MaxPool2d(2)
        #self.conv2 = nn.Conv2d(64, 128, 5)
        #self.conv3 = nn.Conv2d(128, 256, 5)
        
        self.linear1 = nn.Linear(128, 2)
        self.linear2 = nn.Linear(64, 2)
        
        self.dropout = nn.Dropout(0.1)
        
        #self.linear2 = nn.Linear(512, 2)
        
    def forward(self, data):
        res = []
        for i in range(data.shape[1]):
            x = data[:, i]
            x = self.linear1_1(x.view(-1, specsize))
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.linear1_2(x)
            x = F.relu(x)
            #x = self.linear1_3(x)
            #x = F.relu(x)
            res.append(x)
        
#         x1 = data[:,0]
#         x1 = self.linear1_1(x1.view(-1, specsize))
#         x1 = F.relu(x1)
#         x1 = F.dropout(x1, training=self.training)
        
#         x1 = self.linear1_2(x1)
#         x1 = F.relu(x1)
        
#         x1 = self.linear1_3(x1)
#         x1 = F.relu(x1)
        
# #         x1 = self.linear1_3(x1)
# #         x1 = F.relu(x1)
        
# #         x1 = self.linear1_4(x1)
# #         x1 = F.relu(x1)
        
#         x2 = data[:,1]
        
#         x2 = self.linear1_1(x2.view(-1, specsize))
#         x2 = F.relu(x2)
#         x2 = F.dropout(x2, training=self.training)
        
#         x2 = self.linear1_2(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_3(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_3(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_4(x2)
#         x2 = F.relu(x2)
        #res = F.pairwise_distance(x1, x2)
        #res = torch.abs(x1 - x2)
        #print(res.shape)
        #res = self.linear1(res)
        #res = self.linear2(res)
        
        #return F.log_softmax(res, 1)
        return res
        
#        for i in range(2): # Siamese nets; sharing weights
#            x = data[i]
#            x = self.conv1(x)
#            x = F.relu(x)
#            x = self.pool1(x)
#            x = self.conv2(x)
#            x = F.relu(x)
#            x = self.conv3(x)
#            x = F.relu(x)
#            
#            x = x.view(x.shape[0], -1)
#            x = self.linear1(x)
#            res.append(F.relu(x))
#        
#        res = torch.abs(res[1] - res[0])
#        res = self.linear2(res)
#        return res
    def name(self):
        return "Net"

In [275]:
#model = torch.load('siamese99.1%modsworking.pt')
model = torch.load('siamese99.1%modsworking.pt')
model.eval()

Net(
  (linear1_1): Linear(in_features=8000, out_features=1024, bias=True)
  (linear1_2): Linear(in_features=1024, out_features=512, bias=True)
  (linear1_3): Linear(in_features=1024, out_features=512, bias=True)
  (linear1_4): Linear(in_features=512, out_features=256, bias=True)
  (lstm2_1): Linear(in_features=8000, out_features=1024, bias=True)
  (linear2_1): Linear(in_features=1024, out_features=512, bias=True)
  (linear2_2): Linear(in_features=512, out_features=256, bias=True)
  (linear1): Linear(in_features=128, out_features=2, bias=True)
  (linear2): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.1)
)

In [276]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')
batch_size = 1024
#print(spectra[:,0].reshape(len(spectra[:,0]), 1).shape)
print(len(sortedspectra))
spectraTensor = torch.tensor(sortedspectra, dtype=torch.float)[:, None, :]
tspectraTensor = torch.tensor(sortedtspecs, dtype=torch.float)[:, None, :]

spectra_loader = torch.utils.data.DataLoader(dataset=spectraTensor, batch_size=batch_size, shuffle=False)
tspectra_loader = torch.utils.data.DataLoader(dataset=tspectraTensor, batch_size=batch_size, shuffle=False)

99999


In [277]:
#torch.save(spectra_loader, 'spectraloader.pt')
#torch.save(tspectra_loader, 'tspectraloader.pt')

In [278]:
def runModel(loader):
    with torch.no_grad():
        accurate_labels = 0
        all_labels = 0
        loss = 0
        out = []
        for batch_idx, data in enumerate(loader):

            data = data.to(device)    
            out.extend(model(data)[0].cpu().detach().numpy())
    print(len(out))
    return out

In [279]:
spectra_out = runModel(spectra_loader)
tspectra_out = runModel(tspectra_loader)
print(len(spectra_out))
print(len(tspectra_out))
#spectraTensor = model(spectraTensor)[0].cpu().detach().numpy()
#tspectraTensor = model(tspectraTensor)[0].cpu().detach().numpy()
print(tspectraTensor.shape)

99999
88212
99999
88212
torch.Size([88212, 1, 8000])


In [280]:
def find_range(arr, val, tolerance=3):
    assert len(arr) > 0
    assert tolerance > 0
    assert val > 0
    
    left = val - tolerance if val - tolerance >= 0 else 0
    right = val + tolerance
    
    ileft = bisect.bisect_left(arr, left)
    iright = bisect.bisect_right(arr, right)
    
    return (ileft, iright-1) if iright - ileft > 0 else (-1, -1)
    
# Function to insert element 
def insert(lst, n): 
    assert not lst or len(lst[0]) == len(n)
    
    index = num_psm
    # Searching for the position 
    for i in range(len(lst)): 
        if lst[i][0] > n[0]: 
            index = i 
            break
      
    # Inserting n in the list 
    lst = lst[:index] + [n] + lst[index:] 
    return lst[:num_psm]

In [281]:
testmass = sortedspecmasses[3]
left, right = find_range(sortedpepmasses, testmass, precursor_tolerance)
print(left)
print(right)
print(sortedspecmasses[3])
print(sortedpepmasses[2])
lst = [[0.5, 1], [0.7, 2], [0.8, 4]]
lst = []
print(lst)
lst = insert(lst, [0.6, 3])
print(lst)
lst = insert(lst, [0.9, 9])
print(lst)
lst = insert(lst, [0.4, 23])
print(lst)
lst = insert(lst, [0.65, 12])
print(lst)
lst = insert(lst, [0, 7])
print(lst)

54737
73399
1752.898
596.253162
[]
[[0.6, 3]]
[[0.6, 3]]
[[0.4, 23]]
[[0.4, 23]]
[[0, 7]]


In [282]:
psm_scores = []
prev = 0
for ind, spec in enumerate(spectra_out):
    spec_mass = sortedspecmasses[ind]
    left, right = find_range(sortedpepmasses, spec_mass, precursor_tolerance)
    if left != -1 and right != -1:
        candidate_tspecs = tspectra_out[left:right+1]
        l_scores = np.linalg.norm(np.subtract(candidate_tspecs, spec), axis=1)**2
        psm_scores.append(left + l_scores.argsort()[:num_psm])
        #print(ind)
        new = int((ind/len(spectra_out)) * 100)
        if new > prev:
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new
    else:
        print('***********error**************')
        print('ind: ' + str(ind))
        print('spec_mass: ' + str(spec_mass))
        print('**********end error************')

99%


In [283]:
sortedpeps = np.array(sortedpeps)
correct_matches = 0
for ind, psms in enumerate(psm_scores):
    #print('ind: ' + str(ind) + ', mass: ' + str(sortedspecmasses[ind]))
    #print('psms: ' + str(psms) + ', psm mass: ' + str(sortedpepmasses[psms[0]]))
    cand_peps = sortedpeps[psms]
    #print(cand_peps)
    for pep in cand_peps:
        #print('pepidmass: ' + str(pepidmass[pep][0]))
        #print('ids: ' + str(ids[ind]) + ', in: ' + str(pepidmass[pep][0]))
        if ids[ind] in pepidmass[pep][0]:
            correct_matches += 1
            break
print(len(psm_scores))
print(correct_matches)
print(str((correct_matches/len(psm_scores)) * 100) + '% correct matches in top ' + str(num_psm) + ' psms.')

99999
69381
69.38169381693817% correct matches in top 1 psms.


## Accuracy
### Without Mods:
#### Top 5:
With 3 Da precursor tolerance: 99.5%  
With 50 Da precursor tolerance: 96.3%  
With all to all search: 87.9%  
  
#### Top 1:
With 3 Da precursor tolerance: 96.6% in top 1  
With 50 Da precursor tolerance: 89.1% in top 1  
With all to all search 76.0% in top 1
  
### With Mods:
#### Top 5:
82.7%  
  
#### Top 1:
69.4%

In [ ]:
print(count)

In [ ]:
x = [1, 2, 3]
print(*x)
y = [4, 5, 6]
z = list(zip(x, y))
print(z)

In [ ]:
lst = [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]
lstar = np.array(lst)
print(lstar)
x = [1,2,3]
lstar = np.subtract(lstar, x)
print(lstar)
lstar = np.linalg.norm(lstar, axis=1)
print(lstar)
print(lstar**2)

In [ ]:
lst = [0.7, 0.3, 0.5, 0.9]
lst = np.array(lst)
print(lst.argsort())